# Graph database example with Azure OpenAI and Neo4J

You should create a freen account of Neo4J:
https://neo4j.com/

Then you can get access to the console:
https://console.neo4j.io/

<img src="img/NEO4J1.jpg">

In [1]:
#%pip install neo4j

In [2]:
import os
import sys
import openai

from dotenv import load_dotenv
from langchain.chains import GraphCypherQAChain
from langchain.chains import RetrievalQA
from langchain_community.graphs import Neo4jGraph
from langchain_core.documents import Document
from langchain_community.vectorstores import Neo4jVector
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings
from neo4j import GraphDatabase

In [3]:
print(f"Python version: {sys.version}")
print(f"OpenAI version: {openai.__version__}")

Python version: 3.10.11 (main, May 16 2023, 00:28:57) [GCC 11.2.0]
OpenAI version: 1.35.1


## Settings

In [4]:
load_dotenv("azure.env")

# NEO4J
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

# Azure OpenAI
azure_endpoint = os.getenv("AOAI_ENDPOINT")
api_key = os.getenv("AOAI_KEY")
api_version = os.getenv("AOAI_VERSION")

# AOAI models
embedding_model = "text-embedding-ada-002"
model = "gpt-4o"
azure_deployment = "gpt-4o"

In [5]:
os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD

## Azure OpenAI LLM

In [6]:
llm = AzureChatOpenAI(
    model=model,
    azure_deployment=azure_deployment,
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    openai_api_version=api_version,
    temperature=0,
)

In [7]:
#initializing the LLMTransformer model
llm_transformer = LLMGraphTransformer(llm=llm)

## Text to analyse

In [8]:
with open('movie.txt', 'r') as file:
    # Read the entire content of the file
    text = file.read()

print(text)

In Marseille, a police detective follows Alain Charnier, who runs a heroin-smuggling syndicate. The policeman is murdered by Charnier's hitman, Pierre Nicoli. Charnier plans to smuggle $32 million worth of heroin into the United States by hiding it in the car of his unsuspecting friend, television personality Henri Devereaux, who is traveling to New York by ship. In New York City, detectives Jimmy "Popeye" Doyle and Buddy "Cloudy" Russo go out for drinks at the Copacabana. Popeye notices Salvatore "Sal" Boca and his young wife, Angie, entertaining mobsters involved in narcotics. They tail the couple and establish a link between the Bocas and lawyer Joel Weinstock, a buyer in the narcotics underworld. Popeye learns that a shipment of heroin will arrive soon. The detectives convince their supervisor to wiretap the Bocas' phones. Popeye and Cloudy are joined by federal agents Mulderig and Klein.

Devereaux's vehicle arrives in New York City. Boca is anxious to make the purchase while Wein

## Loading the text into the graph database

In [9]:
documents = [Document(page_content=text)]
graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [10]:
# List of nodes
print(f"Nodes:{graph_documents[0].nodes}")

Nodes:[Node(id='Marseille', type='Place'), Node(id='Police Detective', type='Person'), Node(id='Alain Charnier', type='Person'), Node(id='Heroin-Smuggling Syndicate', type='Organization'), Node(id='Pierre Nicoli', type='Person'), Node(id='Henri Devereaux', type='Person'), Node(id='United States', type='Place'), Node(id='New York City', type='Place'), Node(id='Jimmy "Popeye" Doyle', type='Person'), Node(id='Buddy "Cloudy" Russo', type='Person'), Node(id='Copacabana', type='Place'), Node(id='Salvatore "Sal" Boca', type='Person'), Node(id='Angie Boca', type='Person'), Node(id='Mobsters', type='Person'), Node(id='Joel Weinstock', type='Person'), Node(id='Mulderig', type='Person'), Node(id='Klein', type='Person'), Node(id='Washington D.C.', type='Place'), Node(id='Policeman', type='Person'), Node(id='Motorman', type='Person'), Node(id='Train Conductor', type='Person'), Node(id='Lincoln', type='Object'), Node(id='Wards Island', type='Place'), Node(id='Factory', type='Place'), Node(id='Chemis

In [11]:
# List of relationships
print(f"Relationships:{graph_documents[0].relationships}")

Relationships:[Relationship(source=Node(id='Police Detective', type='Person'), target=Node(id='Alain Charnier', type='Person'), type='FOLLOWS'), Relationship(source=Node(id='Alain Charnier', type='Person'), target=Node(id='Heroin-Smuggling Syndicate', type='Organization'), type='RUNS'), Relationship(source=Node(id='Pierre Nicoli', type='Person'), target=Node(id='Police Detective', type='Person'), type='KILLS'), Relationship(source=Node(id='Alain Charnier', type='Person'), target=Node(id='Henri Devereaux', type='Person'), type='FRIEND'), Relationship(source=Node(id='Henri Devereaux', type='Person'), target=Node(id='United States', type='Place'), type='TRAVELS_TO'), Relationship(source=Node(id='Henri Devereaux', type='Person'), target=Node(id='New York City', type='Place'), type='TRAVELS_TO'), Relationship(source=Node(id='Jimmy "Popeye" Doyle', type='Person'), target=Node(id='Buddy "Cloudy" Russo', type='Person'), type='PARTNER'), Relationship(source=Node(id='Jimmy "Popeye" Doyle', type=

In [12]:
graph = Neo4jGraph(url=NEO4J_URI,
                   username=NEO4J_USERNAME,
                   password=NEO4J_PASSWORD)

In [13]:
graph.add_graph_documents(graph_documents,
                          baseEntityLabel=True,
                          include_source=True)

## NEO4J user interface

<img src="img/NEO4J2.jpg">

## Simple test

In [14]:
driver = GraphDatabase.driver(uri=NEO4J_URI,
                              auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

session = driver.session()
result = session.run("MATCH (n) RETURN n LIMIT 1")

for record in result:
    print(record)

<Record n=<Node element_id='4:4760478b-f0be-4b53-a5ac-9ef6e94ab354:0' labels=frozenset({'Document'}) properties={'id': '54f735cde0eddbe8eaa5c96c2faed08b', 'text': 'In Marseille, a police detective follows Alain Charnier, who runs a heroin-smuggling syndicate. The policeman is murdered by Charnier\'s hitman, Pierre Nicoli. Charnier plans to smuggle $32 million worth of heroin into the United States by hiding it in the car of his unsuspecting friend, television personality Henri Devereaux, who is traveling to New York by ship. In New York City, detectives Jimmy "Popeye" Doyle and Buddy "Cloudy" Russo go out for drinks at the Copacabana. Popeye notices Salvatore "Sal" Boca and his young wife, Angie, entertaining mobsters involved in narcotics. They tail the couple and establish a link between the Bocas and lawyer Joel Weinstock, a buyer in the narcotics underworld. Popeye learns that a shipment of heroin will arrive soon. The detectives convince their supervisor to wiretap the Bocas\' pho

## Running the embeddings

In [15]:
embeddings = AzureOpenAIEmbeddings(
    model=embedding_model,
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    openai_api_version=api_version,
)

In [16]:
vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding")

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated procedure. ('db.create.setVectorProperty' has been replaced by 'db.create.setNodeVectorProperty')} {position: line: 1, column: 70, offset: 69} for query: "UNWIND $data AS row MATCH (n:`Document`) WHERE elementId(n) = row.id CALL db.create.setVectorProperty(n, 'embedding', row.embedding) YIELD node RETURN count(*)"


## Testing

In [17]:
query = "Who is Popeye?"

results = vector_index.similarity_search(query, k=1)
print(results[0].page_content)


text: In Marseille, a police detective follows Alain Charnier, who runs a heroin-smuggling syndicate. The policeman is murdered by Charnier's hitman, Pierre Nicoli. Charnier plans to smuggle $32 million worth of heroin into the United States by hiding it in the car of his unsuspecting friend, television personality Henri Devereaux, who is traveling to New York by ship. In New York City, detectives Jimmy "Popeye" Doyle and Buddy "Cloudy" Russo go out for drinks at the Copacabana. Popeye notices Salvatore "Sal" Boca and his young wife, Angie, entertaining mobsters involved in narcotics. They tail the couple and establish a link between the Bocas and lawyer Joel Weinstock, a buyer in the narcotics underworld. Popeye learns that a shipment of heroin will arrive soon. The detectives convince their supervisor to wiretap the Bocas' phones. Popeye and Cloudy are joined by federal agents Mulderig and Klein.

Devereaux's vehicle arrives in New York City. Boca is anxious to make the purchase whi

## GraphCyper QA chain

In [18]:
chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True)

response = chain.invoke({"query": "travels to?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (a)-[:TRAVELS_TO]->(b)
RETURN a, b

Full Context:
[{'a': {'id': 'Alain Charnier'}, 'b': {'id': 'Washington D.C.'}}, {'a': {'id': 'Alain Charnier'}, 'b': {'id': 'Wards Island'}}, {'a': {'id': 'Alain Charnier'}, 'b': {'id': 'Factory'}}, {'a': {'id': 'Alain Charnier'}, 'b': {'id': 'France'}}, {'a': {'id': 'Henri Devereaux'}, 'b': {'id': 'United States'}}, {'a': {'id': 'Henri Devereaux'}, 'b': {'id': 'New York City'}}, {'a': {'id': 'Salvatore "Sal" Boca'}, 'b': {'id': 'Washington D.C.'}}]

> Finished chain.


{'query': 'travels to?',
 'result': 'Alain Charnier travels to Washington D.C., Wards Island, Factory, and France. Henri Devereaux travels to the United States and New York City. Salvatore "Sal" Boca travels to Washington D.C.'}

## Examples of Retrieval QA

In [19]:
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vector_index.as_retriever())

In [20]:
result = qa_chain.invoke({"query": "What is Popeye job?"})
print(result["result"])

Popeye is a police detective.


In [21]:
result = qa_chain.invoke({"query": "What happened in Bay 50Th Street?"})
print(result["result"])

At Bay 50th Street, Popeye Doyle chases Pierre Nicoli, who has just attempted to shoot him. Nicoli boards an elevated train, and Popeye commandeers an automobile to pursue him. During the chase, Nicoli shoots a policeman on the train who tries to intervene and hijacks the motorman at gunpoint, forcing him to drive through the next station and shooting the train conductor. The motorman has a heart attack, and the train is about to crash into a stationary train when an emergency brake engages, hurling Nicoli to the floor. Popeye arrives at Bay 50th Street to see Nicoli descending from the platform. Nicoli sees Popeye, turns to run, but is shot dead by Popeye.


In [22]:
result = qa_chain.invoke({"query": "What happened to Pierre Nicoli?"})
print(result["result"])

Pierre Nicoli was shot dead by Popeye. After a chase that involved Nicoli hijacking an elevated train and causing chaos, Popeye arrived at the scene just as Nicoli was descending from the platform. Nicoli saw Popeye, turned to run, but was shot and killed by Popeye.


In [23]:
result = qa_chain.invoke(
    {"query": "What is the relation between Popeye and the mafia?"})
print(result["result"])

Popeye, whose full name is Jimmy "Popeye" Doyle, is a police detective in New York City. His relation to the mafia is adversarial; he is actively working to investigate and disrupt their narcotics operations. Specifically, he notices Salvatore "Sal" Boca and his wife Angie entertaining mobsters involved in narcotics, which leads him to tail the couple and establish a link between them and lawyer Joel Weinstock, a buyer in the narcotics underworld. Popeye's efforts are focused on uncovering and stopping the mafia's heroin smuggling activities.


In [24]:
result = qa_chain.invoke(
    {"query": "What is the relation between M. Boca and Angie?"})
print(result["result"])

M. Boca, whose full name is Salvatore "Sal" Boca, is married to Angie Boca.


In [25]:
result = qa_chain.invoke(
    {"query": "What is the relation between Angie, M. Boca and Popeye?"})
print(result["result"])

Angie Boca is the wife of Salvatore "Sal" Boca. Popeye, whose real name is Jimmy Doyle, is a police detective who, along with his partner Buddy "Cloudy" Russo, is investigating the Bocas for their involvement in narcotics. Popeye notices Angie and Sal Boca entertaining mobsters and establishes a link between them and the narcotics underworld, leading to further surveillance and investigation.


In [26]:
result = qa_chain.invoke({"query": "Who is the driver of the lincoln car?"})
print(result["result"])

The driver of the Lincoln car is Alain Charnier.


In [27]:
result = qa_chain.invoke(
    {"query": "Display any facts between Popeye and Pierre Nicoli"})
print(result["result"])

Here are the key facts involving Popeye and Pierre Nicoli:

1. **Murder Attempt**: Nicoli attempts to kill Popeye but misses.
2. **Chase**: After the failed assassination attempt, Popeye chases Nicoli, who boards an elevated train.
3. **Train Hijacking**: Nicoli hijacks the train, shooting a policeman and the train conductor, and forces the motorman to drive through the next station.
4. **Confrontation**: The train eventually stops due to an emergency brake, and Nicoli descends from the platform.
5. **Final Encounter**: Popeye arrives, sees Nicoli, and shoots him dead as Nicoli tries to run away.


In [28]:
result = qa_chain.invoke(
    {"query": "List the places with the city and the country"})
print(result["result"])

Sure, here are the places mentioned along with their respective cities and countries:

1. **Marseille, France**
2. **New York City, United States**
3. **Washington D.C., United States**
4. **Grand Central Station, New York City, United States**
5. **Wards Island, New York City, United States**


In [29]:
result = qa_chain.invoke(
    {"query": "List all the people names and their relationship"})
print(result["result"])

Here are the names of the people mentioned in the context along with their relationships:

1. **Alain Charnier** - Runs a heroin-smuggling syndicate.
2. **Pierre Nicoli** - Charnier's hitman.
3. **Henri Devereaux** - Television personality and unsuspecting friend of Charnier.
4. **Jimmy "Popeye" Doyle** - New York City detective.
5. **Buddy "Cloudy" Russo** - New York City detective and partner of Popeye.
6. **Salvatore "Sal" Boca** - Involved in narcotics, linked to Charnier.
7. **Angie Boca** - Sal's young wife.
8. **Joel Weinstock** - Lawyer and buyer in the narcotics underworld.
9. **Mulderig** - Federal agent.
10. **Klein** - Federal agent.
11. **Lou Boca** - Sal's brother and an accessory to the handoff.


In [30]:
result = qa_chain.invoke(
    {"query": "Summarize this in French en quelques lignes"})
print(result["result"])

À Marseille, un détective suit Alain Charnier, chef d'un réseau de trafic d'héroïne, mais est tué par le tueur à gages de Charnier, Pierre Nicoli. Charnier prévoit de faire passer de l'héroïne aux États-Unis en la cachant dans la voiture de son ami Henri Devereaux. À New York, les détectives Jimmy "Popeye" Doyle et Buddy "Cloudy" Russo découvrent le lien entre les Bocas et le trafiquant Joel Weinstock. Après une série de poursuites et de filatures, ils trouvent l'héroïne cachée dans la voiture de Devereaux. Une confrontation finale a lieu, où plusieurs criminels sont arrêtés ou tués, mais Charnier s'échappe. Popeye et Cloudy sont ensuite réaffectés hors de la division des stupéfiants.


In [31]:
result = qa_chain.invoke({
    "query":
    "What is the name, the director of this movie and the release date?"
})
print(result["result"])

The movie is "The French Connection," directed by William Friedkin. It was released on October 7, 1971.


In [32]:
result = qa_chain.invoke(
    {"query": "What are the top 5 movies made by William Friedkin?"})
print(result["result"])

William Friedkin is a renowned director known for several critically acclaimed films. Here are five of his most notable movies:

1. **The French Connection (1971)** - This crime thriller, which you provided context for, won five Academy Awards, including Best Picture and Best Director for Friedkin. It is famous for its intense car chase scene.

2. **The Exorcist (1973)** - A landmark in the horror genre, this film about a young girl possessed by a demonic entity is considered one of the scariest movies of all time. It earned Friedkin an Academy Award nomination for Best Director.

3. **Sorcerer (1977)** - A remake of the French film "The Wages of Fear," this thriller about four men transporting unstable dynamite through a South American jungle has gained a cult following over the years.

4. **To Live and Die in L.A. (1985)** - This neo-noir crime thriller is known for its gritty portrayal of the Los Angeles underworld and features another memorable car chase scene.

5. **Bug (2006)** -

In [33]:
result = qa_chain.invoke({"query": "Give me a list of similar movies"})
print(result["result"])

If you enjoyed the plot of the movie described, which appears to be "The French Connection," you might like the following films that share similar themes of crime, police investigations, and thrilling chases:

1. **Serpico (1973)** - A true story about an honest New York cop who exposes corruption within the police force.
2. **Bullitt (1968)** - Features a San Francisco cop who is determined to find the underworld kingpin that killed the witness in his protection.
3. **Heat (1995)** - A crime drama about a group of professional bank robbers who start to feel the heat from the LAPD when they unknowingly leave a clue at their latest heist.
4. **Dirty Harry (1971)** - Follows a tough San Francisco cop who is determined to track down a psychopathic rooftop killer.
5. **The Departed (2006)** - An undercover cop and a mole in the police force try to identify each other while infiltrating an Irish gang in Boston.
6. **L.A. Confidential (1997)** - A neo-noir crime film about three very differe

In [34]:
result = qa_chain.invoke({"query": "Generate keywords and hashtags"})
print(result["result"])

**Keywords:**
- Marseille
- Police detective
- Alain Charnier
- Heroin smuggling
- Pierre Nicoli
- Henri Devereaux
- New York City
- Jimmy "Popeye" Doyle
- Buddy "Cloudy" Russo
- Salvatore "Sal" Boca
- Angie Boca
- Joel Weinstock
- Wiretap
- Federal agents
- Grand Central Station
- Washington D.C.
- Elevated train chase
- Police stakeout
- Hidden heroin
- Wards Island
- Factory shootout
- Abandoned bakery
- Mulderig
- Narcotics division

**Hashtags:**
- #HeroinSmuggling
- #PoliceDetective
- #CrimeThriller
- #DrugCartel
- #UndercoverOperation
- #NYCDetectives
- #HighStakesChase
- #Wiretap
- #GrandCentralStation
- #ElevatedTrain
- #HiddenDrugs
- #FactoryShootout
- #NarcoticsDivision
- #CrimeDrama
- #Suspense
- #LawEnforcement
- #DrugBust
- #ThrillerMovie
